# Question.1

## (i)
Using lagrange multiplier,
$$
min - \sum \log x_i, \quad \text{s.t.} \Sigma x_i = 1
$$
Denote 
$$
L = - \sum \log x_i + \lambda(\Sigma x_i - 1)
$$
Apply F.O.C

$$
\frac{\partial L}{\partial x_i} = 0 \Rightarrow -\frac{1}{x_i} + \lambda = 0 \Rightarrow x_i = \frac 1\lambda
$$
$$
\frac{\partial L}{\partial \lambda} = 0 \Rightarrow \Sigma x_i = 1 \Rightarrow x_i = \frac 1n, \lambda = n
$$

## (ii)
Similar to (i), denote that 
$$
L = -\sum\log x_i + (Ax - b)^T\lambda
$$
Apply F.O.C,

Here, $A_{ji}$ is the jth row, ith col element in A.
$$
\frac{\partial L}{\partial x_i} = 0 \Rightarrow  -\frac{1}{x_i} + \lambda\sum_j A_{ji} = 0
$$
$$
\frac{\partial L}{\partial\lambda} = 0 \Rightarrow Ax = b.
$$
From the first equaiton, we find that 
$$x_i = \frac{1}{\lambda\sum_jA_{ji}} = \frac{1}{(A^T\lambda)_i}$$.

Also we have,
$$Ax = 
A\left(\begin{array}{c}\frac{1}{\lambda \sum_j A_{j1}}\\
\vdots\\
\frac{1}{\lambda \sum_j A_{jn}}\\
\end{array}\right) = 
\left(
\begin{array}{c}\sum_i\frac{ A_{1i}}{\lambda \sum_j A_{ji}}\\
\vdots\\
\sum_i\frac{ A_{ni}}{\lambda \sum_j A_{ji}}\\
\end{array}\right)
$$
Thus we have
$$
x^TA^T \lambda = \left(
\begin{array}{c}\sum_i\frac{ A_{1i}}{\lambda \sum_j A_{ji}}&
\cdots&
\sum_i\frac{ A_{ni}}{\lambda \sum_j A_{ji}}\\
\end{array}\right)
\left(\begin{array}{cc}\lambda\\ 
\vdots\\
\lambda \end{array}\right) = n
$$
Thus we get the dual larange function
$$
q(\lambda) = \inf_{\lambda}( -\sum\log x_i + (Ax - b)^T\lambda) = -b^T\lambda - \sum_{i = 1}^{n} \frac{1}{(A^T\lambda)_i} + n
$$
i.e.
$$
\max q(\lambda) = -b^T\lambda +\sum_{i = 1}^{n} {(A^T\lambda)_i} + n\qquad \text{where } \lambda > 0.
$$

This is the dual problem.


## (iii)
Already solved in (ii)

## (iv)
By newton's method, we need to calculate the matrix equation
$$
\begin{bmatrix}
\nabla^2 f &A^T\\
A& 0
\end{bmatrix}
\begin{bmatrix}
-\Delta x\\
\lambda
\end{bmatrix}
=
\begin{bmatrix}
-\nabla f(x)\\
0
\end{bmatrix}
$$
It's easy to see that 

$$
\nabla f = (-\frac{1}{x_1},\cdots, -\frac{1}{x_n})^T
$$

$$
\nabla^2 f = diag(\frac{1}{x_1^2},\cdots, \frac{1}{x_n^2})
$$

In [5]:
import numpy as np
from numpy.linalg import inv
def NewtonMethod_acp(A, x0, alpha, beta, tol):
    def decrement(dfx, H):
        # define the function to calculate lambda(x), which is the Newton decrement
        return np.sqrt(np.dot(dfx.T,np.dot(np.diag(1/np.diag(H)), dfx)))
    (row, col) = A.shape
    x = x0
    H = np.diag(1/x**2) # H is hessian of f(x)
    upper = np.column_stack((H, A.T)) # [Hessian, A^T]
    lower = np.column_stack((A, np.zeros((row, row)))) # [A, 0]
    KKT = np.row_stack((upper, lower)) # this the the matrix on the left
    dfx = -1 / x # this is - nabla f(x)
    RHS = np.append(dfx, np.zeros(row)) # right hand side
    epsilon =  decrement(dfx, H) # get the Newton decrement
    count = 0
    while epsilon > tol and count < 500:
        
        count = count + 1
        LHS = np.squeeze(np.array(inv(KKT).dot(RHS)))
        dx = -np.array(LHS)[:col]
        t = 1
        # using backtracking line search
        back_ct=0
        while -np.log(x + t * dx).sum() >= -np.log(x).sum() - alpha * t * dfx.T.dot(dx):
            t = t * beta
            back_ct+=1
            if back_ct>50:
                break
        # update the parameters
        x = x + t * dx
        H = np.diag(1/x**2)
        upper = np.column_stack((H, A.T))
        lower = np.column_stack((A, np.zeros((row, row))))
        KKT = np.row_stack((upper, lower))
        dfx = -1 / x
        RHS = np.append(dfx, np.zeros(row))
        epsilon =  decrement(dfx, H)
    return x,epsilon,count

In [28]:
def NewtonMethod_acp_new_decrement(A, x0, alpha, beta, tol):
    (row, col) = A.shape
    x = x0
    H = np.diag(1/x**2) # H is hessian of f(x)
    upper = np.column_stack((H, A.T)) # [Hessian, A^T]
    lower = np.column_stack((A, np.zeros((row, row)))) # [A, 0]
    KKT = np.row_stack((upper, lower)) # this the the matrix on the left
    dfx = -1 / x # this is - nabla f(x)
    RHS = np.append(dfx, np.zeros(row)) # right hand side
    epsilon =  100 # get the Newton decrement
    count = 0
    while epsilon > tol and count < 500:
        
        count = count + 1
        LHS = np.squeeze(np.array(inv(KKT).dot(RHS)))
        dx = -np.array(LHS)[:col]
        t = 1
        # using backtracking line search
        back_ct=0
        while -np.log(x + t * dx).sum() >= -np.log(x).sum() - alpha * t * dfx.T.dot(dx):
            t = t * beta
            back_ct+=1
            if back_ct>50:
                break
        # update the parameters
        last_x=x
        x = x + t * dx
        H = np.diag(1/x**2)
        upper = np.column_stack((H, A.T))
        lower = np.column_stack((A, np.zeros((row, row))))
        KKT = np.row_stack((upper, lower))
        dfx = -1 / x
        RHS = np.append(dfx, np.zeros(row))
        epsilon =  2*np.sqrt(-np.log(last_x).sum()+np.log(x).sum())
    return x,epsilon,count

In [30]:
a = 0.1
b = 0.5
tol = 0.0001

In [31]:
x=np.array([0.1,0.4,0.3])
A=np.array([[1,1,1]])

In [32]:
NewtonMethod_acp(A, x, a, b, tol)

(array([0.26666667, 0.26666667, 0.26666667]), 1.7320508075688772, 500)

In [33]:
NewtonMethod_acp_new_decrement(A, x, a, b, tol)

(array([0.26666667, 0.26666667, 0.26666667]), 4.398875281426429e-05, 5)

In [10]:
x**2

array([0.01, 0.16, 0.09])

In [35]:
A1 = np.array([1 for i in range(500)])

In [36]:
def cst(j,i):
    if j==i-1 or j==i:
        return 1
    else:
        return 0

In [37]:
A2=np.array([[cst(j,i) for j in range(500)] for i in range(1,100)])

In [38]:
A=np.row_stack((A1,A2))

In [39]:
A

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
x0 = np.array([0.5*i+1 for i in range(500)])

In [41]:
x0

array([  1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,   4.5,   5. ,
         5.5,   6. ,   6.5,   7. ,   7.5,   8. ,   8.5,   9. ,   9.5,
        10. ,  10.5,  11. ,  11.5,  12. ,  12.5,  13. ,  13.5,  14. ,
        14.5,  15. ,  15.5,  16. ,  16.5,  17. ,  17.5,  18. ,  18.5,
        19. ,  19.5,  20. ,  20.5,  21. ,  21.5,  22. ,  22.5,  23. ,
        23.5,  24. ,  24.5,  25. ,  25.5,  26. ,  26.5,  27. ,  27.5,
        28. ,  28.5,  29. ,  29.5,  30. ,  30.5,  31. ,  31.5,  32. ,
        32.5,  33. ,  33.5,  34. ,  34.5,  35. ,  35.5,  36. ,  36.5,
        37. ,  37.5,  38. ,  38.5,  39. ,  39.5,  40. ,  40.5,  41. ,
        41.5,  42. ,  42.5,  43. ,  43.5,  44. ,  44.5,  45. ,  45.5,
        46. ,  46.5,  47. ,  47.5,  48. ,  48.5,  49. ,  49.5,  50. ,
        50.5,  51. ,  51.5,  52. ,  52.5,  53. ,  53.5,  54. ,  54.5,
        55. ,  55.5,  56. ,  56.5,  57. ,  57.5,  58. ,  58.5,  59. ,
        59.5,  60. ,  60.5,  61. ,  61.5,  62. ,  62.5,  63. ,  63.5,
        64. ,  64.5,

In [42]:
res=NewtonMethod_acp(A, x0, a, b, tol)

In [43]:
res

(array([  1.25,   1.25,   2.25,   2.25,   3.25,   3.25,   4.25,   4.25,
          5.25,   5.25,   6.25,   6.25,   7.25,   7.25,   8.25,   8.25,
          9.25,   9.25,  10.25,  10.25,  11.25,  11.25,  12.25,  12.25,
         13.25,  13.25,  14.25,  14.25,  15.25,  15.25,  16.25,  16.25,
         17.25,  17.25,  18.25,  18.25,  19.25,  19.25,  20.25,  20.25,
         21.25,  21.25,  22.25,  22.25,  23.25,  23.25,  24.25,  24.25,
         25.25,  25.25,  26.25,  26.25,  27.25,  27.25,  28.25,  28.25,
         29.25,  29.25,  30.25,  30.25,  31.25,  31.25,  32.25,  32.25,
         33.25,  33.25,  34.25,  34.25,  35.25,  35.25,  36.25,  36.25,
         37.25,  37.25,  38.25,  38.25,  39.25,  39.25,  40.25,  40.25,
         41.25,  41.25,  42.25,  42.25,  43.25,  43.25,  44.25,  44.25,
         45.25,  45.25,  46.25,  46.25,  47.25,  47.25,  48.25,  48.25,
         49.25,  49.25,  50.25,  50.25, 150.75, 150.75, 150.75, 150.75,
        150.75, 150.75, 150.75, 150.75, 150.75, 150.75, 150.75, 

In [44]:
NewtonMethod_acp_new_decrement(A, x0, a, b, tol)

(array([  1.25,   1.25,   2.25,   2.25,   3.25,   3.25,   4.25,   4.25,
          5.25,   5.25,   6.25,   6.25,   7.25,   7.25,   8.25,   8.25,
          9.25,   9.25,  10.25,  10.25,  11.25,  11.25,  12.25,  12.25,
         13.25,  13.25,  14.25,  14.25,  15.25,  15.25,  16.25,  16.25,
         17.25,  17.25,  18.25,  18.25,  19.25,  19.25,  20.25,  20.25,
         21.25,  21.25,  22.25,  22.25,  23.25,  23.25,  24.25,  24.25,
         25.25,  25.25,  26.25,  26.25,  27.25,  27.25,  28.25,  28.25,
         29.25,  29.25,  30.25,  30.25,  31.25,  31.25,  32.25,  32.25,
         33.25,  33.25,  34.25,  34.25,  35.25,  35.25,  36.25,  36.25,
         37.25,  37.25,  38.25,  38.25,  39.25,  39.25,  40.25,  40.25,
         41.25,  41.25,  42.25,  42.25,  43.25,  43.25,  44.25,  44.25,
         45.25,  45.25,  46.25,  46.25,  47.25,  47.25,  48.25,  48.25,
         49.25,  49.25,  50.25,  50.25, 150.75, 150.75, 150.75, 150.75,
        150.75, 150.75, 150.75, 150.75, 150.75, 150.75, 150.75, 

# Question.2

## (a)

First, show the equivelent between these 4 conditions.
___
* Condition 1 is equivalent to Condition 2

Condition 1 $\rightarrow$ Condition 2

Suppose $x \in \mathcal N(A) \cap \mathcal N(H)$ and $x \neq 0$. In condition 2, we have  $Ax = 0\, x \neq 0$. Since $x \in \mathcal N(A) \cap \mathcal N(H)$, we have $x^THx = 0$, this contradict to to second statement.

Condition 2 $\rightarrow$ Condition 1

If there is an $x$ s.t. $Ax = 0\, x \neq 0, x^THX = 0$. Because $H$ is semipositive definate, we must have $Hx = 0$, i.e.   $x \in \mathcal N(A) \cap \mathcal N(H)$
___
* Condition 2 is equivalent to Condition 3

If $Ax = 0, x \neq 0$, then because $\mathcal R(F) = \mathcal N (A)$, we have $x = Fz$ and $z \neq 0$. Then because of 2 we have $x^T Hx = z^TF^THFZ > 0$

___
* Condition 2 is equivalent to Condition 4
If we have the condition 2. then 

$$
x^T(H + A^TA) = x^THx + \vert\vert A^Tx\vert\vert^2_2 > 0
$$
for all nonzero x, sot the condition 4 holds for $Q = I$.
If the fourth condition hols with general semipositive definate form of $Q$.
$$
x^T(H + A^TQA) = x^THx + x^TA^TQAx > 0
$$
for all nonzero $x$, Therefore if $Ax = 0\, x \neq 0$, we must have $x\neq 0$
___
Second show these four conditions are equivalent to nonsigularity of KKT matrix.

Suppose $x\neq 0$, s.t.$Ax = 0, Hx = 0$
$$
\begin{bmatrix}
H & A^T\\
A & 0
\end{bmatrix}
\begin{bmatrix}
x\\
0
\end{bmatrix} = 0$$

If the KKT matrix is singular, $x,z$ are not both zero, and we have
$$
\begin{bmatrix}
H & A^T\\
A & 0
\end{bmatrix}
\begin{bmatrix}
x\\
z
\end{bmatrix} = 0$$

This leads to $Hx + A^Tz = 0$, and $Ax = 0$. 
from the first equation, we have 
$$
x^THx + x^TA^Tz = 0.
$$
plug in the second equation.
we have $x^THx = 0$, this contradicts second condition unless $x= 0$.

## (b)
From previous question, we have $H + A^TA$ is positive definate matrix. Then we must hvae $R \in \mathbb R^{n\times n}$, s.t.
$$
R^T(H + A^TA)R = I
$$
Apply SVD to $AR$, $AR = U\Sigma V_1^T$, where $\Sigma = diag(\sigma_1,\cdots, \sigma_p)$. Denote $V_2 \in \mathbb R^{n\times(n-p)}$, s.t.
$$
V = \left[V_1\quad V_2\right]
$$
$V_2$ is orthogonal, and let 
$$
S = \left[\Sigma\quad 0\right] \in \mathbb R^{p\times n}
$$
We have $AR = USV^T$, then 
$$
V^TR^T(H + A^TA)RV = V^TR^THRV + S^TS = I
$$
Becasue $V^TR^THRV = I - S^TS$ is diagonal matrix, then denote it as $D$
$$
D = V^TR^THRV = diag(1-\sigma_1^2, \cdots, 1-\sigma_p^2, 1, \cdots, 1)
$$
Apply the transform to KKT matrix, we have

$$
\begin{bmatrix}
V^TR^T & 0\\
0 & U
\end{bmatrix}
\begin{bmatrix}
H & A^T\\
A & 0
\end{bmatrix}
\begin{bmatrix}
RV & 0\\
0 & U
\end{bmatrix}
=
\begin{bmatrix}
D & S^T\\
S & 0
\end{bmatrix}
$$
Apply the permutation to hte matrix on the right gives a block diaonal matrix with n diagonal blocks.
$$
\begin{bmatrix}
\lambda_i & \sigma_i\\
\sigma_i & 0
\end{bmatrix}
$$
where $i = 1,2,\cdots, p$, 

and $\lambda_i = 1$ for $i = p +1, \cdots, n$

This matrix have eiggenvalues of $\frac{\lambda_i \pm \sqrt{\lambda_i^2 + 4\sigma_i^2}}{2}$, i.e. one eigen value is positive, another is negative. 

In total, there are n positive evalues and p negative evalues.